# Imports

In [19]:
import cv2
import numpy as np
from deepface import DeepFace

from collections import namedtuple

In [3]:
font = namedtuple('Font', 'pos font fontScale color thickness')\
        ((50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

In [ ]:
# control vars
DEBUG_MODE = False

CAP_FACE_PROPORTION = 1.2
CAP_POSITION = 1.5

XOF = 0
YOF = .8

In [117]:
cap = cv2.imread('images/cap.png', cv2.IMREAD_UNCHANGED)
cap_prop = len(cap[0])/len(cap)

vid = cv2.VideoCapture(0)

while(True):
    
	ret, frame = vid.read()
	if not ret: break

	faces = DeepFace.extract_faces(frame, enforce_detection=False, detector_backend='mtcnn')

	for face in faces:

		x = face["facial_area"]["x"]
		y = face["facial_area"]["y"]
		w = face["facial_area"]["w"]
		h = face["facial_area"]["h"]

		leye = face["facial_area"]["left_eye"]
		reye = face["facial_area"]["right_eye"]

		confidence = face["confidence"]

		if DEBUG_MODE:
			cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2, cv2.LINE_AA)
			cv2.putText(frame, f"{confidence:.2f}", (x, y), font.font, font.fontScale, font.color, font.thickness, cv2.LINE_AA)

			cv2.circle(frame, leye, 2, (0, 0, 255), -1)
			cv2.circle(frame, reye, 2, (0, 0, 255), -1)
		
		
		cap = cv2.resize(cap, (int(w*CAP_FACE_PROPORTION), int(w*CAP_FACE_PROPORTION*cap_prop)))

		try:
			alpha = cap[:,:,3]
			rgb = cap[:,:,:3]

			y1, y2 = y, y + cap.shape[0]
			x1, x2 = x, x + cap.shape[1]
			roi = frame[y1-int(h*YOF):y2-int(h*YOF), x1-int(w*XOF):x2-int(w*XOF)]

			mask = cv2.merge([alpha, alpha, alpha])/255.0

			roi = roi * (1.0-mask) + rgb * mask
			
			frame[y1-int(h*YOF):y2-int(h*YOF), x1-int(w*XOF):x2-int(w*XOF)] = roi
		
		except: pass
	
	cv2.imshow('Webcam', frame)

	if cv2.waitKey(20) == 27: break

vid.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1